In [ ]:
import warnings
warnings.filterwarnings('ignore')
import locale
locale.getpreferredencoding = lambda x=False: "UTF-8"
! pip install -q transformers datasets evaluate
from transformers import AutoModelForCausalLM, BloomTokenizerFast, set_seed
# device = 'cuda'
import torch
import gc
device = 'cuda'
from datasets import load_dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00


In [ ]:
def get_model(model_name):
  model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
  return model

def tokenize(text):
   return tokenizer.encode(text, return_tensors="pt").to( device)

tokenizer = BloomTokenizerFast.from_pretrained("bigscience/bloom")


tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [ ]:
import numpy as np

def get_class_scores(target_id_mapping, scores):
  target_scores = {target: 0 for target in target_id_mapping.keys()}
  for score in scores:
    for target in target_scores.keys():
       target_scores[target] += score[0][target_id_mapping[target]]
  return target_scores

def get_classification(model, example, target_id_mapping):
    text = example.get('text')
    inputs = tokenizer.encode(text, return_tensors="pt").to(device)
    outputs = model.generate(inputs, return_dict_in_generate=True, min_new_tokens=1, max_new_tokens=1, output_scores=True)
    generated_tokens = outputs.sequences[0][inputs.shape[1]:]
    response = tokenizer.decode(generated_tokens)
    target_scores = get_class_scores(target_id_mapping, outputs.scores)
    example['prediction'] = max(target_scores, key= lambda x: target_scores[x])
    example['response'] = response
    return example

def classify_tweets(model, dataset, targets):
    target_ids = [tokenize(target)[0][0] for target in targets]
    return dataset.map(lambda example: get_classification(model, example, {target: target_ids[i] for i, target in enumerate(targets)}))

In [ ]:
models = ['bigscience/bloomz-1b7', 'bigscience/bloom-1b7', 'alonzogarbanzo/Bloom-1b7-winograd-wsc-IT-baseline', 'alonzogarbanzo/Bloom-1b7-glue-mrpc-IT-baseline', "alonzogarbanzo/Bloom-1b7-ropes-Cont-IT-Step2", "alonzogarbanzo/Bloom-1b7-glue-mrpc-Cont-IT-Step3", "alonzogarbanzo/Bloom-1b7-dialogsum-Cont-IT-Step4", "alonzogarbanzo/Bloom-1b7-creative-writing-Cont-IT-Step5"]

In [ ]:
import evaluate
def calc_f1_score(dataset):
    predictions = dataset['prediction']
    references = dataset['label']
    f1 = evaluate.load("f1")
    return f1.compute(predictions=predictions, references=references)

def calc_accuracy(dataset):
    predictions = dataset['prediction']
    references = dataset['label']
    accuracy = evaluate.load("accuracy")
    return accuracy.compute(predictions=predictions, references=references)

def calc_precision(dataset):
    predictions = dataset['prediction']
    references = dataset['label']
    precision = evaluate.load("precision")
    return precision.compute(predictions=predictions, references=references)

def calc_recall(dataset):
    predictions = dataset['prediction']
    references = dataset['label']
    recall = evaluate.load("recall")
    return recall.compute(predictions=predictions, references=references)


In [ ]:
import json
def evaluate_models(models, dataset_name, dataset):
  evaluation_data = {model_name: {} for model_name in models}
  evaluation_metrics = {model_name: {} for model_name in models}
  for model_name in models:
    model = get_model(model_name)
    prediction_dataset = classify_tweets(model, dataset, ['0', '1'])
    evaluation_data[model_name]['f1'] = f1 = calc_f1_score(prediction_dataset)
    evaluation_data[model_name]['accuracy'] = accuracy = calc_accuracy(prediction_dataset)
    evaluation_data[model_name]['precision'] = precision = calc_precision(prediction_dataset)
    evaluation_data[model_name]['recall'] = recall = calc_recall(prediction_dataset)
    prediction_dataset.to_csv(f'./glue-mrpc/{model_name[15:]}_{dataset_name}.csv')
    evaluation_metrics[model_name]['f1'] = f1.get('f1')
    evaluation_metrics[model_name]['accuracy'] = accuracy.get('accuracy')
    evaluation_metrics[model_name]['precision'] = precision.get('precision')
    evaluation_metrics[model_name]['recall'] = recall.get('recall')
    del model
    gc.collect()
    torch.cuda.empty_cache()
    print(f'{model_name} {dataset_name} evaluation complete.\n {evaluation_metrics[model_name]}')

    with open(f'./glue-mrpc/evaluation_metrics_{dataset_name}.json', 'w') as f:
      json.dump(evaluation_metrics, f, indent=4)
  return evaluation_data

In [ ]:
def transform_dataset(dataset, transform_function):
  transformed_data = dataset.map(lambda example: transform_function(example))
  return transformed_data

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

def display_cm(dataset, model_name):
  predictions = dataset['prediction']
  references = dataset['label']
  class_labels = [0, 1, 2]
  cm = confusion_matrix(references, predictions, labels=class_labels)
  plt.figure(figsize=(8, 6))
  disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels)
  disp.plot()
  plt.title(f'{model_name} Confusion Matrix')
  plt.show()


In [ ]:
def transform(example):
    print(example)
    example["text"] = f'''Score the semantic siimilarity between two sentences below. If the sentences are equivalent score it as 1. If the sentences are not equivalent score it as 0.
    Sentence 1: {example.get('sentence1')}
    Sentence 2: {example.get('sentence2')}
    Score: '''
    return example

In [ ]:
dataset_name = "glue-mrpc"

In [ ]:
dataset = load_dataset("adambjorn/UnrelatedForgettingOverhead", data_dir=dataset_name, split="test")


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:

dataset = transform_dataset(dataset, transform)
data = evaluate_models(models, dataset_name, dataset)

Map:   0%|          | 0/75 [00:00<?, ? examples/s]

{'sentence1': 'Kids , adults , booksellers and postal workers all are preparing for " Harry Potter and the Order of the Phoenix . "', 'sentence2': 'The crates are full of hardback copies of " Harry Potter and the Order of the Phoenix . "', 'label': 0, 'idx': 415}
{'sentence1': 'The proportion of people covered by employers dropped from 62.3 percent in 2001 to 61.3 percent last year .', 'sentence2': 'The proportion of Americans with insurance from employers declined to 61.3 percent , from 62.6 percent in 2001 and 63.6 percent in 2000 .', 'label': 0, 'idx': 1671}
{'sentence1': 'Retailers J.C. Penney Co . Inc.JCP.N and Walgreen Co.WAG.N kick things off early in the week .', 'sentence2': 'Retailers J.C. Penney Co . Inc . JCP.N and Walgreen Co . WAG.N kick things off on Monday .', 'label': 1, 'idx': 395}
{'sentence1': 'Advanced Micro Devices Inc . Tuesday introduced its Athlon MP 2800 + processor for entry-level one- and two-way servers and workstations .', 'sentence2': 'Advanced Micro Devi

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Map:   0%|          | 0/75 [00:00<?, ? examples/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Map:   0%|          | 0/75 [00:00<?, ? examples/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

alonzogarbanzo/Bloom-1b7-creative-writing-Cont-IT-Step5 glue-mrpc evaluation complete.
 {'f1': 0.24137931034482757, 'accuracy': 0.41333333333333333, 'precision': 0.7777777777777778, 'recall': 0.14285714285714285}


In [ ]:
import os
import zipfile

def zip_directory(folder_path, output_filename):
    # Create a zipfile object in write mode
    with zipfile.ZipFile(output_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # The os.walk function allows us to walk through the directory tree
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                # Create the file path by joining the directory and file name
                file_path = os.path.join(root, file)
                # Create the archive name, which is the path within the zip file
                # This is the path relative to the folder we are zipping
                arcname = os.path.relpath(file_path, os.path.dirname(folder_path))
                # Write the file to the zipfile
                zipf.write(file_path, arcname)

# Example usage
folder_path = './glue-mrpc'  # Replace with the path to your directory
output_filename = 'glue-mrpc.zip'  # Replace with your desired output zip file name
zip_directory(folder_path, output_filename)
